In [3]:
import pandas as pd
import numpy as np

In [4]:
addidas = pd.read_csv('./addidas_full.csv')

In [5]:
addidas.head()

,Unnamed: 0,Order#,Carton#,Units,SKU,SOLD_TO,SHIPTO_ZIP,CREATE_DATE_TIME,DIVISION,Date,Day,Hour,DayofWeek,Week,Week_ending,3ZIP
0,0,E11364536,980900433424831,1,12056838,ECOM,11050,12/28/2016 21:16:02,APP,2016-12-28 21:16:02,2016-12-28,21,Wednesday,52,2016-12-30,110
1,1,E11364542,980900433428570,1,11458754,ECOM,11385,12/28/2016 21:16:02,APP,2016-12-28 21:16:02,2016-12-28,21,Wednesday,52,2016-12-30,113
2,2,E11378556,980900433486211,1,11564403,ECOM,92553,12/29/2016 13:46:02,APP,2016-12-29 13:46:02,2016-12-29,13,Thursday,52,2016-12-30,925
3,3,E11389293,980900433625436,1,10727437,ECOM,7030,12/30/2016 3:58:02,APP,2016-12-30 03:58:02,2016-12-30,3,Friday,52,2017-01-06,70
4,4,E11380841,980900433628093,1,11919739,ECOM,92620,12/29/2016 17:17:55,APP,2016-12-29 17:17:55,2016-12-29,17,Thursday,52,2016-12-30,926


In [ ]:
# new column: region
# r1 = zips 20000
# r2 = zips 20000-40000
# r3 = zips > 40000

In [6]:
len(addidas)

5644365

In [75]:
# Number of orders each SKU is in
orders_in = addidas.groupby('SKU')[['Order#']].count().rename(columns={'Order#':'OrdersIn'})
orders_in.head()

,OrdersIn
SKU,
3857515,1
3857516,1
3857518,1
3857519,1
3857565,1


In [107]:
# Region-based metric
region1_df = addidas[addidas['SHIPTO_ZIP'] < 50000]
r1_total_units = region1_df['Units'].sum()
r1_num_unique_ord = len(region1_df['Order#'].unique())
r1_avg_units = r1_total_units/r1_num_unique_ord


region2_df = addidas[(addidas['SHIPTO_ZIP']>=50000) & (addidas['SHIPTO_ZIP']<80000)]
r2_total_units = region2_df['Units'].sum()
r2_num_unique_ord = len(region2_df['Order#'].unique())
r2_avg_units = r2_total_units/r2_num_unique_ord


region3_df = addidas[addidas['SHIPTO_ZIP'] >= 80000]
r3_total_units = region3_df['Units'].sum()
r3_num_unique_ord = len(region3_df['Order#'].unique())
r3_avg_units = r3_total_units/r3_num_unique_ord


total_unq_ord = r1_num_unique_ord + r2_num_unique_ord + r3_num_unique_ord
total_avg_units = r1_total_units + r2_total_units + r3_total_units


data = {
    'Region':['R1','R2','R3'],
    'Avg Units/Order':[round(r1_avg_units,2), round(r2_avg_units,2), round(r3_avg_units,2)],
    'Total Unique Order (%)':[
        round(r1_num_unique_ord/total_unq_ord,2), 
        round(r2_num_unique_ord/total_unq_ord,2), 
        round(r3_num_unique_ord/total_unq_ord,2)],
    'Total Average Units (%)':[
        round(r1_total_units/total_avg_units,2), 
        round(r2_total_units/total_avg_units,2), 
        round(r3_total_units/total_avg_units,2)]
}

regions_df = pd.DataFrame(data, columns=data.keys())

regions_df

,Region,Avg Units/Order,Total Average Units (%),Total Unique Order (%)
0,R1,1.47,0.49,0.49
1,R2,1.39,0.19,0.19
2,R3,1.50,0.32,0.32


In [108]:
# Region identity
region_identity_df = pd.DataFrame({
    'region':addidas['SHIPTO_ZIP'].astype(str).str.zfill(5),
    'value':None
})


In [109]:
zip_as_int = region_identity_df['region'].astype(np.int64)
region_identity_df.loc[zip_as_int < 50000,'value'] = 0
region_identity_df.loc[(zip_as_int >= 50000) & (zip_as_int < 80000), 'value'] = 1
region_identity_df.loc[zip_as_int >= 80000,'value'] = 2

region_identity_df = region_identity_df.drop_duplicates(subset=['region'])

In [110]:
region_identity_df.head()

,region,value
0,11050,0
1,11385,0
2,92553,2
3,07030,0
4,92620,2


In [111]:
# Create CSVs
regions_df.to_csv("region_metrics.csv", index=False)
region_identity_df.to_csv("region_identity.csv")

In [106]:
len(region_identity_df)

31931